In [3]:
def two_gap(V, delta1 = 2e-4, delta2 = 2e-4, Z1 = 1.0, Z2 = 1.0, broadness1 = 1e-6, broadness2 = 1e-6, w = .5):
    offset = 1.4e-5
    def ingap(E, delta):
        return E <= delta

    def u2(E, delta): #u0 squared, BCS coefficient
        return .5 * (1+np.sqrt((E**2 - delta**2)/E**2))

    def v2(E, delta): # v0 squared, BCS coefficient
        return (1-u2(E, delta))

    def gamma2(E, delta, Z): # gamma squared
        return (u2(E, delta) + (Z**2)*(u2(E, delta) - v2(E, delta)))**2

    def A(E, delta, Z):
        if E <= delta:
            return delta**2/(E**2 + (delta**2 - E**2)*(1+2*Z**2)**2)
        if E > delta:
            return (u2(E, delta)*v2(E, delta))/gamma2(E, delta, Z)

    def B(E, delta, Z):
        if E < delta:
            return 1 - delta**2/(E**2 + (delta**2 - E**2)*(1+2*Z**2)**2)
        if E >= delta:
            return ((u2(E, delta)-v2(E, delta))**2*Z**2*(1+Z**2))/gamma2(E, delta, Z)

    def f0_deriv(expo):
        return ((c/(k*T))*expo)/(expo+1)**2

    def f0(expo):
        return 1/(expo+1)

    def per_e(e, v, delta, Z, broadness):
        expo_f = exp(v)
        expo_f_deriv = exp(e-c*v)
        return (f0_deriv(expo_f_deriv)-f0(expo_f))*(1 + A(e, delta, Z)-B(e, delta, Z))

    def exp(E):
        return np.exp(np.real(E)/(k*T))

    def DIDV(V, delta, Z, broadness):
        return np.sum(np.array([np.real(per_e(e+1j*broadness, V, delta, Z, broadness)) for e in E])  /len(E))

    #Rn = .5*norm(delta)
    #return scale*(1/Rn)*np.array([DIDV(abs(v-offset)) for v in V]) data not well normalized
    norm1, norm2 = 2/DIDV(0, delta1, 0, 0), 2/DIDV(0, delta2, 0, 0)
    return np.array([w * norm1 *DIDV(abs(v-offset), delta1, Z1, broadness1) +
                     (1-w) * norm2 *DIDV(abs(v-offset), delta2, Z2, broadness2) for v in V])

In [2]:
k= 8.617e-5 #eV/K Boltzmann
T= 55e-3 #temp of 1 mK
c=1
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.colors as colors
from scipy.optimize import curve_fit
import time
import smtplib


def mail(subject_matter, body):
    gmail_user = 'pistockmail@gmail.com'
    gmail_password = 'stockpiticker'
    sent_from = gmail_user
    to = 'david@vdriel.nl'
    subject = 'Update about '+str(subject_matter)
    email_text = body
    message = 'Subject: {}\n\n{}'.format(subject, email_text)
    try:
        server = smtplib.SMTP_SSL('smtp.gmail.com', 465)
        server.ehlo()
        server.login(gmail_user, gmail_password)
        server.sendmail(sent_from, to, message)
        server.close()
    except:
        print ('Something went wrong...')

def gap(V, delta = 2e-4, Z = 1.0, broadness = 0.01):
    def ingap(E):
        return E <= delta

    def u2(E): #u0 squared, BCS coefficient
        return .5 * (1+np.sqrt((E**2 - delta**2)/E**2))

    def v2(E): # v0 squared, BCS coefficient
        return (1-u2(E))

    def gamma2(E): # gamma squared
        return (u2(E) + (Z**2)*(u2(E) - v2(E)))**2

    def A(E):
        if E <= delta:
            return delta**2/(E**2 + (delta**2 - E**2)*(1+2*Z**2)**2)
        if E > delta:
            return (u2(E)*v2(E))/gamma2(E)

    def B(E):
        if E < delta:
            return 1 - delta**2/(E**2 + (delta**2 - E**2)*(1+2*Z**2)**2)
        if E >= delta:
            return ((u2(E)-v2(E))**2*Z**2*(1+Z**2))/gamma2(E)

    def tunnel(E): #dI/dV https://www.researchgate.net/publication/46424263_Environment-Assisted_Tunneling_as_an_Origin_of_the_Dynes_Density_of_States
        E += 1j*broadness
        return abs(np.real(E/(((E**2-delta**2))**.5)))

    def tunnel2(E): #dI/dV https://www.researchgate.net/publication/46424263_Environment-Assisted_Tunneling_as_an_Origin_of_the_Dynes_Density_of_States
        E += 1j*broadness
        if -delta<E<delta:
            return abs(np.real(E/(((E**2-delta**2))**.5)))
        else:
            return 0

    def f0_deriv(expo):
        return ((c/(k*T))*expo)/(expo+1)**2

    def f0(expo):
        return 1/(expo+1)

    def per_e(e, v):
        expo_f = exp(v)
        expo_f_deriv = exp(e-c*v)
        return (f0_deriv(expo_f_deriv)-f0(expo_f))*(1+A(e)-B(e))

    def exp(E):
        return np.exp(np.real(E)/(k*T))

    def DIDV(V):
        return np.sum(np.array([np.real(per_e(e + 1j*broadness, V)) for e in E])  /len(E))
    
    def norm_calc():
        def gamma2(E): # gamma squared
            return (u2(E) + (Z**2)*(u2(E) - v2(E)))**2
        
        
        def A(E):
            if E <= delta:
                return delta**2/(E**2 + (delta**2 - E**2)*(1+2*Z**2)**2)
            if E > delta:
                return (u2(E)*v2(E))/gamma2(E)

        def B(E):
            if E < delta:
                return 1 - delta**2/(E**2 + (delta**2 - E**2)*(1+2*Z**2)**2)
            if E >= delta:
                return ((u2(E)-v2(E))**2*Z**2*(1+Z**2))/gamma2(E)
        
        def A_norm(E):
            if E <= delta:
                return delta**2/(E**2 + (delta**2 - E**2))
            if E > delta:
                return (u2(E)*v2(E))/gamma2(E)
        
        def B_norm(E):
            if E < delta:
                return 1 - delta**2/(E**2 + (delta**2 - E**2))
            if E >= delta:
                return 0
        
        def per_e_norm(e, v):

            expo_f = exp(v)
            expo_f_deriv = exp(e-c*v)
            return (f0_deriv(expo_f_deriv)-f0(expo_f))*(1+A(e)-B(e))
        
        #return 2/(np.sum(np.array([np.real(per_e_norm(e, 0)) for e in E])  /len(E)))
        E = np.arange(-5*delta,5*delta, 1.0000001e-5)
        return 1/(np.sum(np.array([np.real(per_e_norm(e, 2*delta)) for e in E])  /len(E)))

    #norm = 2/DIDV(0)

    return norm_calc() * np.array([DIDV(abs(v)) for v in V])



def dataload(filename = '', column = 0):
    G0 = 12906
    factor = 1e-6 #1mV/V unit is now millivolts
    data = np.loadtxt(filename+".dat")
    lines = open(filename + ".meta.txt").read().splitlines()
    x,y = lines[3], lines[7] #y is fixed as x is sweeped, the y up
    len_x, len_y = int(lines[0]), int(lines[4]) #So, column x sweeped first
    datavalues = [x,y,"z", 'valueK1','valueK2','valueK3','valueL1','valueL2',0,'K1','K2','K3','RL1','thetaL1','RL2','thetaL2',0,0]

    def ind(st):
        return datavalues.index(st)

    def f(n):
        return n*len_x



    if len_y == 1:
        Vbias = factor * data[:, ind(x)] -7272 * data[:, ind('valueK1')]


        g = G0*data[:, ind('valueL1')]
    else:
        BG_datapoint = f(column)
        Vbias = factor * data[BG_datapoint : BG_datapoint + len_x, ind(x)] \
                -7272*data[BG_datapoint : BG_datapoint + len_x, ind('valueK1')]
        g = G0*data[BG_datapoint : BG_datapoint + len_x, ind('valueL1')]
        print(("BG = "+str(data[f(column),ind(y)] )))
    return Vbias, g

Vn, gn = dataload('2.1&3.1_R-M_7', 274)
#V, delta = 2e-4, Z = 1.0, broadness = 0.01
p0 = ( 2.25e-04, .6, 0)
E = np.arange(1.2*min(Vn),1.2*max(Vn), 1.0000001e-5)
start = time.time()
popt = p0
plt.plot(Vn, gap(Vn, *popt), label="Fit")
plt.scatter(Vn, gn, s=1, label="Data")
plt.xlim(-3e-4,3e-4)

plt.title('as')

#plt.plot(Vn, gap(Vn, delta = popt[0], Z = popt[2], broadness = popt[4]), label = "gap1")
#plt.plot(Vn, gap(Vn, delta = popt[1], Z = popt[3], broadness = popt[5]), label = "gap2")
plt.legend()

plt.show()


BG = 378.0


<Figure size 640x480 with 1 Axes>

In [3]:
Vn, gn = dataload('Device_3_2.dat')

OSError: Device_3_2.dat.dat not found.